In [ ]:
import matplotlib.image as mpimg
from matplotlib.pyplot import *
import numpy
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import math

caffeModel = "C:/Users/click/Phyton/DNN_V2/models/res10_300x300_ssd_iter_140000.caffemodel"
prototextPath = "C:/Users/click/Phyton/DNN_V2/models/deploy.prototxt.txt"

In [ ]:
X=300
def face_detection(image):
    net = cv2.dnn.readNetFromCaffe(prototextPath,caffeModel)
    # extract the dimensions , Resize image into 300x300 and converting image into blobFromImage
    (h,w) = image.shape[:2]

    # blobImage convert RGB (104.0, 177.0, 123.0)
    blob = cv2.dnn.blobFromImage(cv2.resize(image,(X,X)),1.0,(X,X),(104.0, 177.0, 123.0))   #,(104.0, 177.0, 123.0)


    #passing blob through the network to detect and pridiction
    net.setInput(blob)
    detections = net.forward()
    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence and prediction
           confidence = detections[0, 0, i, 2]
           #print(confidence)
           if confidence > 0.5:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                text = "{:.2f}%".format(confidence * 100)
                y = startY - 10 if startY - 10 > 10 else startY + 10
                cv2.rectangle(image, (startX, startY), (endX, endY),(0, 0, 255), 2)
                #cv2.putText(image, text, (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
    #plt.imshow( image[startY:endY,startX:endX]) #test face extraction
    #plt.imshow(image)
    #print(image.shape)
    return image[startY:endY,startX:endX]
    

In [ ]:
img=cv2.imread("C:/Users/click/Phyton/DNN_V2/src/dataset/thor.2.1 (6).jpg",cv2.IMREAD_GRAYSCALE)
img3=cv2.equalizeHist(img)
img4=np.zeros((img.shape[0],img.shape[1]))
img4=cv2.normalize(img2,  img4, 0, 255, cv2.NORM_MINMAX)
img2=AllinOne(img)
img5=img4-img3
cv2.imwrite("contrasteDIFF.jpg",img4)

In [ ]:
import cv2
import numpy as np
from PIL import Image
import os
from timeit import default_timer as timer 

# Path for face image database
path = 'LFW/training'

start = timer()
recognizer = cv2.face.LBPHFaceRecognizer_create()
detector = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml");
# function to get the images and label data
def getImagesAndLabels(path):
    d=0
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]
    faceSamples=[]
    ids = []
    for imagePath in imagePaths:
        PIL_img = Image.open(imagePath).convert('L') # grayscale
        img_numpy = np.array(PIL_img,'uint8')
        id = int(os.path.split(imagePath)[-1].split(".")[1])
        #print(id)
        faces = detector.detectMultiScale(img_numpy)
        d+=1
        for (x,y,w,h) in faces:
            face=(cv2.equalizeHist(img_numpy[y:y+h,x:x+w]))
            pth='C:/Users/click/Phyton/DNN_V2/src/verification/verif%d.jpg'%d
            faceSamples.append(face)
            ids.append(id)
            print(d)
            cv2.imwrite(pth,face)
            
    return faceSamples,ids
print ("\n [INFO] Training faces. It will take a few seconds. Wait ...")
faces,ids = getImagesAndLabels(path)
recognizer.train(faces, np.array(ids))
# Save the model into trainer/trainer.yml
recognizer.write('trainer/trainerJ.yml') 
# Print the numer of faces trained and end program
print("\n [INFO] {0} faces trained. Exiting Program".format(len(np.unique(ids))))
print("sec ",round(timer() - start,2))

In [ ]:
import cv2
import numpy as np
from PIL import Image
import os
# chemin de la base d'image
path = 'LFW/training'



recognizer = cv2.face.LBPHFaceRecognizer_create()
detector = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml");
# fontions pour récuperer les noms de fichiers et images
def getImgsAndLabels(path):
    d=0
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]
    faceSamples=[]
    ids = []
    for imagePath in imagePaths:
        PIL_img = cv2.imread(imagePath) # grayscale
        img_numpy = np.array(PIL_img,'uint8')
        id = int(os.path.split(imagePath)[-1].split(".")[1])
        #print(id)
        faces = face_detection(img_numpy)
        d+=1
        print(d)
        print(faces.size)
        if(faces.size>0):  
            
            face=(cv2.cvtColor(faces, cv2.COLOR_BGR2GRAY))
            facef=(cv2.equalizeHist(face))
            pth='C:/Users/click/Phyton/DNN_V2/src/verification/verif%d.jpg'%d
            faceSamples.append(facef)
            ids.append(id)   
            cv2.imwrite(pth,facef)
        
        
            
    return faceSamples,ids
print ("\n [INFO] Training faces. It will take a few seconds. Wait ...")
faces,ids = getImgsAndLabels(path)
recognizer.train(faces, np.array(ids))
# Ssauvegarde du modele
recognizer.write('trainer/trainerLFW.yml') 
# affichage du nombre de visages entrainé  
print("\n [INFO] {0} faces trained. Exiting Program".format(len(np.unique(ids))))

In [ ]:
from re import search
names=['Colin_Powell','Donald_Rumsfeld','George_W_Bush', 'Gerhard_Schroede', 'Hugo_Chavez', 'Tony_Blair']

ide=[]

mypath='C:/Users/click/Phyton/DNN_V2/src/LFW/test'
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]

cp=0
print(names[cp])

for f in range(435):
    print(cp)
    #si le nom d'image ne change pas on ajoute le meme ID
    if(names[cp] in onlyfiles[f]):
        ide.append(cp)
        print(cp)
    #si le nom change alors on incrémente l'ID
    else:
        ide.append(cp)
        cp+=1
        
        
    print(onlyfiles[f],onlyfiles[f+1])
    
        
    
print(ide)
print(len(ide))


In [ ]:
from os import listdir
from os.path import isfile, join
import numpy
import cv2
from glob import glob
import requests
import math
import os 
 


recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainerLFW.yml')


cont=0
pth='C:/Users/click/Phyton/DNN_V2/src/LFW/test'
mypath='C:/Users/click/Phyton/DNN_V2/src/LFW/test'
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
print(onlyfiles)
imgPaths = [os.path.join(pth,f) for f in os.listdir(pth)]
print(len(imgPaths))
faceSamples=[]
rec = []
f=0

names=['none','Colin_Powell','Donald_Rumsfeld','George_W_Bush', 'Gerhard_Schroede', 'Hugo_Chavez', 'Tony_Blair']

ide=[]

print(len(imgPaths))
for imgPath in imgPaths:
        if(f >= 435):
            break
        
        img=cv2.imread(imgPath)
        f+=1
        if(img is not None):
            
            print("and ---->",f)
            
            face=face_detection(img)
            grayFace=cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
            plt.imshow(img)
            id, confid=recognizer.predict(grayFace)
        
            name=names[id]
            print('nom prédit-->',name,' nom exact-->',onlyfiles[f])
            if(name in onlyfiles[f]):
                cont+=1
        
            print(cont)
        

print("la précision du modele est de --->",100*(cont/f),"%")




In [ ]:
import requests
import cv2
import numpy as np
import matplotlib.pyplot as plt
import numpy
import math
import os 
import requests
from timeit import default_timer as timer 


recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('C:/Users/user/LBP/model/trainerJ.yml')

In [ ]:
from re import search

confusion=numpy.zeros((6,6), dtype=np.uint8)
#print(confusion)

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainerLFW.yml')


cont=0
pth='C:/Users/click/Phyton/DNN_V2/src/LFW/test'
mypath='C:/Users/click/Phyton/DNN_V2/src/LFW/test'
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
#print(onlyfiles)
imgPaths = [os.path.join(pth,f) for f in os.listdir(pth)]
print(len(imgPaths))
faceSamples=[]
rec = []
f=0
print(len(onlyfiles))
#définition des noms
names=['Colin_Powell','Donald_Rumsfeld','George_W_Bush', 'Gerhard_Schroede', 'Hugo_Chavez', 'Tony_Blair']
#names=['alan_grant','claire_dearing','ellie_sattler', 'ian_malcolm', 'john_hammond', 'owen_grady']

print("taille",len(ids))

print(len(imgPaths))
#parcours du dossier
for imgPath in imgPaths:
    
    if(f >= 435 ):
        break    
    img=cv2.imread(imgPath)
    f+=1
    print(f)
    face=face_detection(img)
    grayFace=cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
    #prediction de l'image en entrée
    id, confid=recognizer.predict(grayFace)
    name=names[id-1]
    #si le nom de l'image prédite est similaire a celle de l'image de test, on incrémente la diagonale.
    if(name in onlyfiles[f]):
        i=ide[f]
        j=ide[f]
        print("attribution -->",i,j,"--->",confusion[i][j])
        confusion[i][j]+=1
        
    #sinon on récupere l'indice de nom de l'image prédite afin d'incrémenter la case adéquate de l'indivdu courant
    if(name not in onlyfiles[f]):
        #print(name)
        
            
        if(name in names[names.index(name)]):
            v=names.index(name)
            #print(v)
            #print("index image prédite --->",v)
            confusion[i][v]+=1
    
    print(confusion)

           
